In [2]:
"""
README: YOLO Detection vs. Ground Truth Per-Class, Per-City Visualizer

This script compares YOLO detection results to ground-truth labels, per class and per city,
filters the top-10 best-matching images (by IoU threshold) for each class/city combination,
and saves separate image folders for side-by-side visual inspection.
- Also supports visual zoom-ins for small objects.
- Produces 'original.jpg', 'labels.jpg', and 'detections.jpg' for each best match.
- Parallel processing is used for efficiency.

How to use:
- Update the directory paths below for your detection labels, ground truth labels, and images.
- The script will output a folder structure organized by class and city.
- Zoomed crops for small objects are included for easier visual analysis.

Requirements:
- Python 3.7+
- opencv-python
- pathlib (standard)
- numpy
- concurrent.futures (standard)
- datetime (standard)
- time (standard)

Author: Bahadir Akin Akgul
Date: 13.07.2025
"""

import cv2
import time
from pathlib import Path
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

# === SETTINGS ===
labels_dir = Path('PATH_TO_YOLO_DETECTION_LABELS')
gt_dir = Path('PATH_TO_GROUND_TRUTH_LABELS')
images_dir = Path('PATH_TO_IMAGES')
output_dir = Path('yolo-best10-perclass-city-visuals')
output_dir.mkdir(parents=True, exist_ok=True)

iou_threshold = 0.5
class_names = ['pedestrian', 'road', 'vehicle']
class_ids = {'pedestrian': 0, 'road': 1, 'vehicle': 2}
colors = {
    'gt': (0, 255, 0),       # green for ground truth
    0: (0, 255, 255),        # yellow for pedestrian
    1: (0, 0, 0),            # black for road
    2: (0, 0, 255)           # red for vehicle
}

city_keywords = {
    'istanbul': ['libadiye', 'levent', 'taksim', 'ciragan', 'barbaros', 'dolmabahce', 'bagdat', 'muallim', 'katar'],
    'paris': ['paris-champs'],
    'munich': ['munih'],
    'marseille': ['marsilya']
}

city_translation = {
    'istanbul': 'Istanbul',
    'paris': 'Paris',
    'munich': 'Munich',
    'marseille': 'Marseille',
    'unknown': 'Unknown'
}

def now():
    return datetime.now().strftime('%H:%M:%S')

def calculate_iou(boxA, boxB):
    xA, yA, wA, hA = boxA
    xa1, ya1, xa2, ya2 = xA - wA / 2, yA - hA / 2, xA + wA / 2, yA + hA / 2
    xB, yB, wB, hB = boxB
    xb1, yb1, xb2, yb2 = xB - wB / 2, yB - hB / 2, xB + wB / 2, yB + hB / 2

    inter_x1 = max(xa1, xb1)
    inter_y1 = max(ya1, yb1)
    inter_x2 = min(xa2, xb2)
    inter_y2 = min(ya2, yb2)
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    area_a = (xa2 - xa1) * (ya2 - ya1)
    area_b = (xb2 - xb1) * (yb2 - yb1)
    return inter_area / (area_a + area_b - inter_area + 1e-6)

def draw_single_class_boxes(image, boxes, classes, cls_id, color_override=None, is_gt=False):
    h, w = image.shape[:2]
    for cls, box in zip(classes, boxes):
        if cls != cls_id:
            continue
        x_c, y_c, bw, bh = box
        x1 = int((x_c - bw / 2) * w)
        y1 = int((y_c - bh / 2) * h)
        x2 = int((x_c + bw / 2) * w)
        y2 = int((y_c + bh / 2) * h)

        color = color_override if color_override else colors[cls]
        thickness = 1
        if cls_id == 1:  # road
            thickness = 2
        cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
    return image

def check_overlap(new_box, existing_boxes):
    x1, y1, x2, y2 = new_box
    for ex in existing_boxes:
        ex1, ey1, ex2, ey2 = ex
        if not (x2 < ex1 or x1 > ex2 or y2 < ey1 or y1 > ey2):
            return True
    return False

def get_city_from_name(name):
    for city, keywords in city_keywords.items():
        if any(keyword in name for keyword in keywords):
            return city
    return 'unknown'

def process_file_for_class(cls_name):
    start = time.time()
    cls_id = class_ids[cls_name]
    results_by_city = {city: [] for city in city_keywords}
    results_by_city['unknown'] = []
    txt_files = list(labels_dir.glob("*.txt"))

    print(f"[{now()}] 🚀 Starting: {cls_name.upper()}")

    for txt_file in txt_files:
        image_stem = txt_file.stem
        image_file = images_dir / f"{image_stem}.jpg"
        if not image_file.exists():
            continue

        img = cv2.imread(str(image_file))
        if img is None:
            continue

        gt_path = gt_dir / txt_file.name
        if not gt_path.exists():
            continue

        with open(gt_path) as f:
            gt_lines = [line.strip().split() for line in f if line.strip()]
        gt_classes = [int(l[0]) for l in gt_lines]
        gt_boxes = [list(map(float, l[1:5])) for l in gt_lines]

        with open(txt_file) as f:
            det_lines = [line.strip().split() for line in f if line.strip()]
        det_classes = [int(l[0]) for l in det_lines]
        det_boxes = [list(map(float, l[1:5])) for l in det_lines]

        filtered_det_boxes = []
        filtered_det_classes = []
        matched_gt = set()

        for i, gt_box in enumerate(gt_boxes):
            if gt_classes[i] != cls_id:
                continue
            for j, det_box in enumerate(det_boxes):
                if det_classes[j] != cls_id:
                    continue
                if calculate_iou(gt_box, det_box) >= iou_threshold:
                    filtered_det_boxes.append(det_box)
                    filtered_det_classes.append(cls_id)
                    matched_gt.add(i)
                    break

        match_count = len(matched_gt)
        if match_count > 0:
            city = get_city_from_name(image_stem)
            results_by_city[city].append({
                "match": match_count,
                "image": img,
                "gt_boxes": gt_boxes,
                "gt_classes": gt_classes,
                "det_boxes": filtered_det_boxes,
                "det_classes": filtered_det_classes,
                "name": txt_file.stem
            })

    for city, results in results_by_city.items():
        city_label = city_translation.get(city, city.capitalize())
        city_output = output_dir / cls_name / city_label
        city_output.mkdir(parents=True, exist_ok=True)

        top10 = sorted(results, key=lambda x: x["match"], reverse=True)[:10]

        for i, result in enumerate(top10, 1):
            base_name = f"{i:02d}_{result['name']}"
            single_output = city_output / base_name
            single_output.mkdir(parents=True, exist_ok=True)

            ori_img = result["image"]
            gt_img = draw_single_class_boxes(
                ori_img.copy(), result["gt_boxes"], result["gt_classes"],
                cls_id, color_override=colors['gt'], is_gt=True)

            det_img = draw_single_class_boxes(
                ori_img.copy(), result["det_boxes"], result["det_classes"],
                cls_id, color_override=colors[cls_id], is_gt=False)

            h, w = ori_img.shape[:2]
            zoom_overlay = det_img.copy()
            max_zoom_size = 96
            zoom_positions = []

            for k, box in enumerate(result["det_boxes"]):
                x_c, y_c, bw, bh = box
                box_w = int(bw * w)
                box_h = int(bh * h)
                if box_w >= 32 or box_h >= 32:
                    continue

                x1 = max(0, int((x_c - bw / 2) * w))
                y1 = max(0, int((y_c - bh / 2) * h))
                x2 = min(w, int((x_c + bw / 2) * w))
                y2 = min(h, int((y_c + bh / 2) * h))
                crop = ori_img[y1:y2, x1:x2]
                if crop.size <= 0:
                    continue

                zoom_factor = min(max_zoom_size // max(box_w, box_h), 4)
                zoom_w, zoom_h = box_w * zoom_factor, box_h * zoom_factor
                resized = cv2.resize(crop, (zoom_w, zoom_h), interpolation=cv2.INTER_LINEAR)
                cv2.rectangle(resized, (0, 0), (zoom_w - 1, zoom_h - 1), colors[cls_id], 2)

                paste_x, paste_y = x1, y1 - zoom_h - 10
                shift = 5
                while check_overlap((paste_x, paste_y, paste_x + zoom_w, paste_y + zoom_h), zoom_positions):
                    paste_y -= shift
                    if paste_y < 0:
                        paste_y = y2 + 10
                    if paste_y + zoom_h > h:
                        break

                if paste_y + zoom_h > h or paste_x + zoom_w > w:
                    continue

                zoom_positions.append((paste_x, paste_y, paste_x + zoom_w, paste_y + zoom_h))
                zoom_overlay[paste_y:paste_y+zoom_h, paste_x:paste_x+zoom_w] = resized

                arrow_start = (x1 + box_w // 2, y1)
                arrow_end = (paste_x + zoom_w // 2, paste_y + zoom_h - 1)
                cv2.arrowedLine(zoom_overlay, arrow_start, arrow_end, (255, 0, 255), 1, tipLength=0.2)

            cv2.imwrite(str(single_output / "original.jpg"), ori_img)
            cv2.imwrite(str(single_output / "labels.jpg"), gt_img)
            cv2.imwrite(str(single_output / "detections.jpg"), zoom_overlay)

            print(f"[{now()}] 📂 {cls_name.upper()} - {city_label} - Saved: {base_name}")

    print(f"[{now()}] ✅ {cls_name.upper()} finished ({time.time()-start:.1f} sec)")

# Parallel processing per class
with ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(process_file_for_class, class_names)


[20:32:07] 🚀 Başlıyor: PEDESTRIAN
[20:32:07] 🚀 Başlıyor: ROAD
[20:32:07] 🚀 Başlıyor: VEHICLE
[20:32:24] 📂 ROAD - Istanbul - Kaydedildi: 01_katar-482_jpg.rf.a2522fa55e30b6b22de87cdd7b30df63
[20:32:24] 📂 PEDESTRIAN - Istanbul - Kaydedildi: 01_bagdat-1-382_jpg.rf.4ae73d9941bdea605dc2dbd0280ed888
[20:32:24] 📂 ROAD - Istanbul - Kaydedildi: 02_bagdat-1-293_jpg.rf.74450bad7504458a2a6298c5ec855fd1
[20:32:24] 📂 PEDESTRIAN - Istanbul - Kaydedildi: 02_bagdat-1-422_jpg.rf.7e3ee5b10b2373426f0701098ce3a18e
[20:32:24] 📂 ROAD - Istanbul - Kaydedildi: 03_bagdat-1-4_jpg.rf.665736ea603fff5ddba9e99f9fb4ecf7
[20:32:24] 📂 PEDESTRIAN - Istanbul - Kaydedildi: 03_bagdat-1-381_jpg.rf.4ff3273da5bf43f5739f9de23572004d
[20:32:24] 📂 ROAD - Istanbul - Kaydedildi: 04_muallimnaci-284_jpg.rf.dd3b9eca3aaf5054e147bd82721f6121
[20:32:24] 📂 PEDESTRIAN - Istanbul - Kaydedildi: 04_bagdat-1-257_jpg.rf.74ab909f0e2a8232f83b52696f2a6bd3
[20:32:24] 📂 ROAD - Istanbul - Kaydedildi: 05_ciragan-108_jpg.rf.2e2117ac297317c91a05682123ab

In [1]:
import cv2
import time
from pathlib import Path
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from PIL import Image

# === AYARLAR ===
labels_dir = Path('runs/detect/yolov10-txtresults/labels')
gt_dir = Path('/truba/home/baakgul/roadtr-14032025/test/labels')
images_dir = Path('/truba/home/baakgul/roadtr-14032025/test/images')
output_dir = Path('yolov10-best-10-separated-final-300dpi')  # GÜNCELLENDİ
output_dir.mkdir(parents=True, exist_ok=True)

iou_threshold = 0.5
class_names = ['pedestrian', 'road', 'vehicle']
class_ids = {'pedestrian': 0, 'road': 1, 'vehicle': 2}
colors = {
    'gt': (0, 255, 0),
    0: (0, 255, 255),
    1: (0, 0, 0),
    2: (0, 0, 255)
}

city_keywords = {
    'istanbul': ['libadiye', 'levent', 'taksim', 'ciragan', 'barbaros', 'dolmabahce', 'bagdat', 'muallim', 'katar'],
    'paris': ['paris-champs'],
    'munih': ['munih'],
    'marsilya': ['marsilya']
}

city_translation = {
    'istanbul': 'Istanbul',
    'paris': 'Paris',
    'munih': 'Munich',
    'marsilya': 'Marseille',
    'unknown': 'Unknown'
}

def now(): return datetime.now().strftime('%H:%M:%S')

def calculate_iou(boxA, boxB):
    xA, yA, wA, hA = boxA
    xa1, ya1, xa2, ya2 = xA - wA / 2, yA - hA / 2, xA + wA / 2, yA + hA / 2
    xB, yB, wB, hB = boxB
    xb1, yb1, xb2, yb2 = xB - wB / 2, yB - hB / 2, xB + wB / 2, yB + hB / 2

    inter_x1 = max(xa1, xb1)
    inter_y1 = max(ya1, yb1)
    inter_x2 = min(xa2, xb2)
    inter_y2 = min(ya2, yb2)
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    area_a = (xa2 - xa1) * (ya2 - ya1)
    area_b = (xb2 - xb1) * (yb2 - yb1)
    return inter_area / (area_a + area_b - inter_area + 1e-6)

def draw_single_class_boxes(image, boxes, classes, cls_id, color_override=None, is_gt=False):
    h, w = image.shape[:2]
    for cls, box in zip(classes, boxes):
        if cls != cls_id:
            continue
        x_c, y_c, bw, bh = box
        x1 = int((x_c - bw / 2) * w)
        y1 = int((y_c - bh / 2) * h)
        x2 = int((x_c + bw / 2) * w)
        y2 = int((y_c + bh / 2) * h)

        color = color_override if color_override else colors[cls]
        thickness = 2 if cls_id == 1 else 1
        cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
    return image

def check_overlap(new_box, existing_boxes):
    x1, y1, x2, y2 = new_box
    for ex in existing_boxes:
        ex1, ey1, ex2, ey2 = ex
        if not (x2 < ex1 or x1 > ex2 or y2 < ey1 or y1 > ey2):
            return True
    return False

def get_city_from_name(name):
    for city, keywords in city_keywords.items():
        if any(keyword in name for keyword in keywords):
            return city
    return 'unknown'

def save_with_dpi(img_array, path, dpi=(300, 300)):
    img_rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)
    img_pil.save(path, dpi=dpi, quality=95)

def process_file_for_class(cls_name):
    start = time.time()
    cls_id = class_ids[cls_name]
    results_by_city = {city: [] for city in city_keywords}
    results_by_city['unknown'] = []
    txt_files = list(labels_dir.glob("*.txt"))

    print(f"[{now()}] 🚀 Başlıyor: {cls_name.upper()}")

    for txt_file in txt_files:
        image_stem = txt_file.stem
        image_file = images_dir / f"{image_stem}.jpg"
        if not image_file.exists():
            continue

        img = cv2.imread(str(image_file))
        if img is None:
            continue

        gt_path = gt_dir / txt_file.name
        if not gt_path.exists():
            continue

        with open(gt_path) as f:
            gt_lines = [line.strip().split() for line in f if line.strip()]
        gt_classes = [int(l[0]) for l in gt_lines]
        gt_boxes = [list(map(float, l[1:5])) for l in gt_lines]

        with open(txt_file) as f:
            det_lines = [line.strip().split() for line in f if line.strip()]
        det_classes = [int(l[0]) for l in det_lines]
        det_boxes = [list(map(float, l[1:5])) for l in det_lines]

        filtered_det_boxes = []
        filtered_det_classes = []
        matched_gt = set()

        for i, gt_box in enumerate(gt_boxes):
            if gt_classes[i] != cls_id:
                continue
            for j, det_box in enumerate(det_boxes):
                if det_classes[j] != cls_id:
                    continue
                if calculate_iou(gt_box, det_box) >= iou_threshold:
                    filtered_det_boxes.append(det_box)
                    filtered_det_classes.append(cls_id)
                    matched_gt.add(i)
                    break

        match_count = len(matched_gt)
        if match_count > 0:
            city = get_city_from_name(image_stem)
            results_by_city[city].append({
                "match": match_count,
                "image": img,
                "gt_boxes": gt_boxes,
                "gt_classes": gt_classes,
                "det_boxes": filtered_det_boxes,
                "det_classes": filtered_det_classes,
                "name": txt_file.stem
            })

    for city, results in results_by_city.items():
        city_label = city_translation.get(city, city.capitalize())
        city_output = output_dir / cls_name / city_label
        city_output.mkdir(parents=True, exist_ok=True)

        top10 = sorted(results, key=lambda x: x["match"], reverse=True)[:10]

        for i, result in enumerate(top10, 1):
            base_name = f"{i:02d}_{result['name']}"
            single_output = city_output / base_name
            single_output.mkdir(parents=True, exist_ok=True)

            ori_img = result["image"]
            gt_img = draw_single_class_boxes(
                ori_img.copy(), result["gt_boxes"], result["gt_classes"],
                cls_id, color_override=colors['gt'])

            det_img = draw_single_class_boxes(
                ori_img.copy(), result["det_boxes"], result["det_classes"],
                cls_id, color_override=colors[cls_id])

            h, w = ori_img.shape[:2]
            zoom_overlay = det_img.copy()
            max_zoom_size = 96
            zoom_positions = []

            for box in result["det_boxes"]:
                x_c, y_c, bw, bh = box
                box_w = int(bw * w)
                box_h = int(bh * h)
                if box_w >= 32 or box_h >= 32:
                    continue

                x1 = max(0, int((x_c - bw / 2) * w))
                y1 = max(0, int((y_c - bh / 2) * h))
                x2 = min(w, int((x_c + bw / 2) * w))
                y2 = min(h, int((y_c + bh / 2) * h))
                crop = ori_img[y1:y2, x1:x2]
                if crop.size <= 0:
                    continue

                zoom_factor = min(max_zoom_size // max(box_w, box_h), 4)
                zoom_w, zoom_h = box_w * zoom_factor, box_h * zoom_factor
                resized = cv2.resize(crop, (zoom_w, zoom_h), interpolation=cv2.INTER_LINEAR)
                cv2.rectangle(resized, (0, 0), (zoom_w - 1, zoom_h - 1), colors[cls_id], 2)

                paste_x, paste_y = x1, y1 - zoom_h - 10
                while check_overlap((paste_x, paste_y, paste_x + zoom_w, paste_y + zoom_h), zoom_positions):
                    paste_y -= 5
                    if paste_y < 0:
                        paste_y = y2 + 10
                    if paste_y + zoom_h > h:
                        break

                if paste_y + zoom_h > h or paste_x + zoom_w > w:
                    continue

                zoom_positions.append((paste_x, paste_y, paste_x + zoom_w, paste_y + zoom_h))
                zoom_overlay[paste_y:paste_y+zoom_h, paste_x:paste_x+zoom_w] = resized

                arrow_start = (x1 + box_w // 2, y1)
                arrow_end = (paste_x + zoom_w // 2, paste_y + zoom_h - 1)
                cv2.arrowedLine(zoom_overlay, arrow_start, arrow_end, (255, 0, 255), 1, tipLength=0.2)

            save_with_dpi(ori_img, single_output / "original.jpg")
            save_with_dpi(gt_img, single_output / "labels.jpg")
            save_with_dpi(zoom_overlay, single_output / "detections.jpg")

            print(f"[{now()}] 📂 {cls_name.upper()} - {city_label} - Kaydedildi: {base_name}")

    print(f"[{now()}] ✅ {cls_name.upper()} tamamlandı ({time.time()-start:.1f} saniye)")

# === Paralel çalıştır ===
with ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(process_file_for_class, class_names)


[01:23:15] 🚀 Başlıyor: PEDESTRIAN
[01:23:15] 🚀 Başlıyor: VEHICLE
[01:23:15] 🚀 Başlıyor: ROAD
[01:23:50] 📂 ROAD - Istanbul - Kaydedildi: 01_katar-482_jpg.rf.a2522fa55e30b6b22de87cdd7b30df63
[01:23:50] 📂 ROAD - Istanbul - Kaydedildi: 02_bagdat-1-293_jpg.rf.74450bad7504458a2a6298c5ec855fd1
[01:23:50] 📂 ROAD - Istanbul - Kaydedildi: 03_bagdat-1-4_jpg.rf.665736ea603fff5ddba9e99f9fb4ecf7
[01:23:50] 📂 PEDESTRIAN - Istanbul - Kaydedildi: 01_bagdat-1-382_jpg.rf.4ae73d9941bdea605dc2dbd0280ed888
[01:23:50] 📂 ROAD - Istanbul - Kaydedildi: 04_muallimnaci-284_jpg.rf.dd3b9eca3aaf5054e147bd82721f6121
[01:23:50] 📂 PEDESTRIAN - Istanbul - Kaydedildi: 02_bagdat-1-422_jpg.rf.7e3ee5b10b2373426f0701098ce3a18e
[01:23:51] 📂 ROAD - Istanbul - Kaydedildi: 05_ciragan-108_jpg.rf.2e2117ac297317c91a05682123abc5a0
[01:23:51] 📂 PEDESTRIAN - Istanbul - Kaydedildi: 03_bagdat-1-381_jpg.rf.4ff3273da5bf43f5739f9de23572004d
[01:23:51] 📂 ROAD - Istanbul - Kaydedildi: 06_katar-1104_jpg.rf.35de596930f413624c5a3d3038216dfb
[01